In [ ]:
# Read file
library("survey")
library("sas7bdat") 
# 15624 x 239
#AA = read.sas7bdat("C:/Users/Rex/Desktop/OWH/yrbs2015.sas7bdat")
#save(AA, file="C:/Users/Rex/Desktop/OWH/yrb2015.Rdata")
load("C:/Users/Rex/Desktop/OWH/yrb2015.Rdata")
"file loaded"


In [ ]:
paste. = function(...,sep='') paste(...,sep=sep)

tidyresult = function(res) {
    foo = c(mean(res),SE(res), confint(res))
    names(foo)  = NULL
    foo
}
    
cc = function(v) {
    s = paste0("c(",v[1])
    v = v[-1]
    for (x in v) s = paste0(s,",",x)
    return(paste0(s,")"))
}
    
yrbssCalculate =
  function (
    locale = 'national',
      year = '2015',
    # Responses to be reported:
    responses = list(Q8 = c(2, 4), Q9 = c(1, 2)),
    # Filters (e.g. demographic) to be applied to respondents:
    filter = list(Q1 = c(4, 5, 6), # age 15-18
                  Q2 = a(1),
                  # female
                  Q5 = a(3)),  # black
    
      
    
      
    group_by = list("Q3","Q4"),
    # List of variables that responses should be grouped by.
    # style of confidence interval; argumment to svyciprop.
    confint = c("xlogit",
                "logit",
                "mean",
                "beta")
  )
  {
    D = AA
    design = svydesign(ids=~PSU,
                 strata=~STRATUM,
                 weights=~WEIGHT,
                 data=D)
      
      validate.column.name = function(var) {
          
          if (!(var %in% colnames(D))) {print("Bad column name")}
      }
      

    #load("bigfile.Rda")  # bigdata 
    #D = bigdata[[locale]]
    #if (D==NULL) somekindoferror(locale)
    ### for each filter condition
    cond = TRUE
    for (var in names(filter)) {
        validate.column.name(var)
        cond = cond & (D[[var]] %in% filter[[var]])
    }
    cat(sum(cond), "passed filters.\n")
    
    all_by_vals = lapply(group_by, function(s) sort(unique(D[[s]])))
    names(all_by_vals) = group_by
    grid = expand.grid(all_by_vals)
    #print(grid)
    assigned = 0
    ### for each cell in grid
    for (i in 1:nrow(grid)) {
        newcond = cond
        for (s in group_by) {
            val = grid[i,s]
            #print(c(s,val))
            newcond = newcond & (D[[s]] == val)
        }
        cat( sum(newcond), "filtered and in grid entry.\n")
        assigned = assigned + sum(newcond)
        
        ### for each question
        for (resp.name in names(responses)) {
            cat("\nResponse",resp.name,"\n")
            resp.true = D[[resp.name]] %in% responses[[resp.name]]
            cat("Grid:\n")
            print(grid[i,])
            #cat(sum(resp.true), "interesting responses.\n")
            #cat(sum(resp.true & newcond), "interesting responses in this grid cell.\n")
            #cat(sum(newcond), "subjects in this grid cell.\n")
            prop = sum(resp.true & newcond) / sum(newcond)
            count = sum(!is.na(D[[resp.name]]) & newcond)
            cat("Count", count, "\n")
            if (count == 0) next
            cat("Proportion", prop, "\n")
            
            ss = subset(design,newcond)
            bar = paste.("svyciprop(~(",resp.name,"%in%",cc(responses[[resp.name]]),"),",
                        "ss, method='xlogit', na.rm=TRUE)")
            print(bar)
            print(round(100*tidyresult(eval(parse(text=bar))),2))

            
        }
    }     
    cat(assigned, "assigned to grid.\n")
    cat("\n") 
  }
      

    yrbssCalculate(filter=list(QN8=1:2))
    yrbssCalculate(filter=list(QN9=c(1,3,5)))
    yrbssCalculate(filter=list(QN8=1:2, QN9=c(1,3,5)))
    yrbssCalculate(filter=list(Q1=c(4, 5, 6), # age 15-18
                              Q2=c(1), # female
                              Q5=c("C","E")))

In [ ]:
# Attach level to demographic columns
levels(AA$Q1) = c("","<=12","13","14","15","16","17",">=18")
levels(AA$Q2) = c("","Female","Male")
levels(AA$Q3) = c("","9th","10th","11th","12th","Other")
levels(AA$Q4) = c("","Yes","No")
levels(AA$RACEETH) =  c("","AI/AN","Asian","Black-African-American","NH/PI","White",
                        "Hispanic/Latino","Multiple-Hispanic","Multiple-Non-Hispanic")
"OK"

In [ ]:
D = AA

if (FALSE) {# Eliminate rows with missing demographic data.
    b1 = AA$Q1=="" #age
    b2 = AA$Q2=="" #sex
    b3 = AA$Q3=="" #grade
    b4 = AA$RACEETH==""
    b5 = AA$Q3=="Other"
    c(sum(b1),sum(b2),sum(b3),sum(b4),sum(b5))
    b = b1|b2|b3|b4|b5
    sum(b)
        b = !b
    # 15118 x 239
    D = AA[b|TRUE,]
}

# D = D[!is.na(D$QN8),]
dim(D)
D$QN8 = 2 - D$QN8
D$QN9 = 2 - D$QN9


In [ ]:
# Turn demographic columns in to factors.
D$Q1 = factor(as.character(D$Q1))
D$Q2 = factor(as.character(D$Q2))
D$Q3 = factor(as.character(D$Q3))
D$Q4 = factor(as.character(D$Q4))
D$RACEETH = factor(as.character(D$RACEETH))
D$Q3 = relevel(D$Q3,ref="9th")


In [ ]:
# Label columns.
names(D)[1:4] = c("Age","Sex","Grade","Eth")
#names(D)[233] = "Race"
names(D)[237] = "Race"
names(D)

In [ ]:
yrbs = svydesign(ids=~PSU,
                 strata=~STRATUM,
                 weights=~WEIGHT,
                 data=D)
# Stratified 1 - level Cluster Sampling design (with replacement)
# With (54) clusters.
# svydesign(ids = ~PSU, strata = ~STRATUM, weights = ~WEIGHT, data = D)

if(FALSE) {
    svymean(~QN8,yrbs)
svymean(~QN8,subset(yrbs,Sex=='Female'))
svymean(~QN8,subset(yrbs,Sex=='Male'))
svymean(~QN8,subset(yrbs,Race=='Asian'))
svymean(~QN8,subset(yrbs,Sex=='Male' & Race=='Asian'))
svymean(~QN8,subset(yrbs,Sex=='Female' & Race=='White'))
svymean(~QN8,subset(yrbs,Sex=='Female' & Race=='White' & Grade=='10th'))
svymean(~QN8,subset(yrbs,Race=='Hispanic/Latino'))
    }

handle_condition = function(D,g,r,s) {
    condition = TRUE
    if (g != 'Total') {
        condition = condition & D$Grade==g
    }
    if (r != 'Total') {
        condition = condition & D$Race==r
    }
    if (s != 'Total') {
        condition = condition & D$Sex==s
    }
        
    
    print(paste(g,r,s))
    cat(file=f,paste(paste(g,r,s), ','))
    ss = subset(yrbs,condition)
    ss2 = subset(yrbs, condition & !is.na(D$QN8))
    print(nrow(ss2))
    cat(file=f,nrow(ss2),',')
    if (nrow(ss2)==0) {
        print("N/A")
        cat(file=f,"NA,")
    } else {
        for (method in c("xlogit")){  #,"logit","asin", "beta","mean")) {
            foo = tidyresult(svyciprop(~QN8, ss, method=method, na.rm=TRUE))
            print(foo)
            foo = round(100*foo,2)
            cat(file=f,paste(foo[1],',',foo[2],',',foo[3],',',foo[4],'\n'))
        }
    }
    cat("\n")

        
}

tidyresult = function(res) {
    foo = c(mean(res),SE(res), confint(res))
    names(foo)  = NULL
    foo
}

tidyratioresult = function(result) {
    foo = c(result[[1]][1], SE(result)[1], confint(result)[1,])
    names(foo) = NULL
    foo
}

tidycontrastresult = function(res) {
    foo = c(res[1],SE(res),confint(res))
    names(foo) = NULL
    foo
}

f = file("foo.csv","w")
for (g in c('Total','9th', '10th','11th','12th')) {        
    for (r in c('Total','Asian','Black-African-American', 'White')) {
        for (s in c('Total','Female','Male')) {
           # for (a in c('14','15','16')) {
                  handle_condition(D,g,r,s)
         #   }
        }
    }
}
close(f)

In [ ]:
cp = function(L) {
    L[[1]]
}

list()
L3 = list( c('a','b','c'), 1:3, c('x','y','z'))

In [ ]:
NULL

In [ ]:
length(list())

In [ ]:
list()==list()


In [ ]:
D


In [ ]:
getwd()


In [ ]:
library("sas7bdat") 
setwd('../OWH/')

sadc.2015.national.df = read.sas7bdat("sadc_2015_national.sas7bdat")
print("ok")

save("sadc.2015.national.df", file="sadc_2015_national.rda")
print("done")

In [6]:
library("sas7bdat") 
setwd('../OWH/')
load('sadc_2015_national.rda')

In [8]:
load('sadc_2015_state_a_m.rda')

In [10]:
load('sadc_2015_state_n_z.rda')

In [14]:
load('sadc_2015_national.rda')
dim(sadc.2015.national.df)

[1] 188898    287

In [27]:
load('sadc_2015_state_a_m.rda')
dim(sadc.2015.state.a.m.df)

[1] 511110    287

In [17]:
load('sadc_2015_state_n_z.rda')
dim(sadc.2015.state.n.z.df)

[1] 431705    287

In [23]:
load('sadc_2015_district.rda')
dim(sadc.2015.district.df)

[1] 310476    287

In [24]:
?memset

In [26]:
object.size(sadc.2015.state.a.m.df)

936978464 bytes